In [2]:
import pandas as pd
from modules.lavenshtein import lavenshtein, get_laven
import os
from multiprocessing import Pool

In [25]:
users_data = pd.read_csv("data/interim/Users.csv").head(10)
with Pool() as p:
    result = p.map(get_laven, users_data.to_dict('records'))
result_df = pd.DataFrame(result)


pot_doc        [00d0164592408, 00d0530724847, 00d0114666542, ...
pot_name       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
pot_price      [2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
pot_date       [7, 7, 8, 7, 8, 7, 8, 7, 7, 8, 8, 7, 6, 7, 8, ...
pot_address    [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...
dtype: object
pot_doc        []
pot_name       []
pot_price      []
pot_date       []
pot_address    []
dtype: object
pot_doc        []
pot_name       []
pot_price      []
pot_date       []
pot_address    []
dtype: object
pot_doc        []
pot_name       []
pot_price      []
pot_date       []
pot_address    []
dtype: object
pot_doc        [00d0624439175, 00d0876488640, 00d0948483091, ...
pot_name                          [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
pot_price                         [2, 2, 3, 2, 2, 3, 3, 3, 3, 2]
pot_date                          [6, 8, 9, 7, 7, 8, 7, 8, 8, 7]
pot_address             [12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
dtype: object
pot_doc   

In [26]:

# final_df = result_df

# final_df = pd.concat([final_df.reset_index(drop=True),users_data["documentid"].reset_index(drop=True)],axis = 1)
# final_df.to_csv(os.path.join('data/final/', "final.csv"))
# display(final_df)
# final_df.to_csv(os.path.join('data/final/', "sfefw.csv"))
display(result_df)
for i in result_df['pot_doc']:
    print(len(i))


,pot_doc,pot_name,pot_price,pot_date,pot_address
0,"[00d0624439175, 00d0876488640, 00d0948483091, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[3, 4, 3, 3, 2, 3, 3, 3, 4, 3]","[7, 8, 8, 7, 7, 8, 6, 8, 8, 7]","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
1,[],[],[],[],[]
2,"[00d0734493835, 00d0546591203, 00d0750315464, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 2, 3, 3, ...","[7, 7, 7, 7, 6, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, ...","[4, 5, 4, 4, 4, 4, 5, 5, 4, 4, 5, 4, 4, 4, 4, ..."
3,[],[],[],[],[]
4,[],[],[],[],[]
5,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[3, 2, 3, 3, 3, 3, 1, 1, 1, 3, 1, 1, 2, 1, 1, ...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[16, 16, 16, 15, 16, 16, 16, 15, 15, 15, 16, 1..."
6,"[00d0164592408, 00d0530724847, 00d0114666542, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[7, 7, 8, 7, 8, 7, 8, 7, 7, 8, 8, 7, 6, 7, 8, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
7,"[00d0624439175, 00d0876488640, 00d0948483091, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[2, 2, 3, 2, 2, 3, 3, 3, 3, 2]","[6, 8, 9, 7, 7, 8, 7, 8, 8, 7]","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
8,[],[],[],[],[]
9,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2, 2, 1, 3, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 3, ...","[8, 8, 8, 8, 8, 8, 7, 7, 7, 8, 7, 8, 8, 7, 8, ...","[16, 16, 16, 15, 16, 16, 16, 15, 15, 15, 16, 1..."


10
0
20
0
0
38
24
10
0
38


In [22]:
# new_df = final_df.apply(lambda row: (row['documentid'] in row['doc_id1'])
# | (row['documentid'] in row['doc_id2'])
# | (row['documentid'] in row['doc_id3'])
# | (row['documentid'] in row['doc_id4'])
# | (row['documentid'] in row['doc_id5'])
# , axis=1)
# #display(new_df)
# print(new_df.value_counts()[True])

65
